In [ ]:
import os
os.chdir('/home/orenmil/raid/mds/240801_standalone_blood_aging/code')
# print(os.getcwd())


import sys
import shutil
# caution: path[0] is reserved for script path (or '' in REPL)

import importlib
import itertools
import metacells as mc
import anndata as ad
import pandas as pd
import numpy as np
import scipy.cluster.hierarchy
import scipy.spatial.distance
import matplotlib
import seaborn as sb
import matplotlib.pyplot as plt
import pathlib
import logging
import re
import collections
import sklearn
import sklearn.compose
import pickle

# from generic import gene_module_utils
from generic import generic_utils
from generic import ml_utils
from mds import mds_analysis_params
from mds import mds_analysis
# from mds import ipssm_utils
from generic import mc_utils
from mds import arch_mutation_interface_and_utils
from mds import clinical_data_interface_and_utils
from sc_rna_seq_preprocessing import sc_rna_seq_preprocessing_params

plt.rcParams["patch.force_edgecolor"] = False
plt.rcParams['patch.linewidth'] = 0
plt.rcParams['patch.edgecolor'] = 'none'
# plt.rcParams['scatter.edgecolors'] = 'black' # didnt affect sb.scatterplot

%matplotlib widget
%load_ext autoreload
%autoreload 2

def get_mds_params():
    return mds_analysis_params.MDS_ANALYSIS_PARAMS


def get_sc_rna_seq_preprocessing_params():
    return sc_rna_seq_preprocessing_params.SC_RNA_SEQ_PREPROCESSING_PARAMS

plt.subplots()
plt.close('all')

In [ ]:
# 

mds_analysis.assert_multi_sample_donors_have_explicit_exp_names_in_cna_infos()

list_of_dataset_name_and_ext_feature_df_csv_file_paths = [
        ('ref', mds_analysis_params.get_mc_model_paths('final_normal_pb_atlas', dataset_name='illu_mds')['ext_donor_feature_df']),
        ('illu', mds_analysis_params.get_mc_model_paths('final_mds_cyto_normal_excluding_atlas', dataset_name='illu_mds')['ext_donor_feature_df']),
        ('ult', mds_analysis_params.get_mc_model_paths('final_mds_cyto_normal_excluding_atlas', dataset_name='ult_mds')['ext_donor_feature_df']),
]

print(f'the last modification time of the input files are:')
for _, x in list_of_dataset_name_and_ext_feature_df_csv_file_paths:
    print(f'{x}: {generic_utils.get_file_last_modification_time_str_repr(x)}')
all_ext_donor_feature_df = pd.concat([pd.read_csv(x[1]).assign(dataset_name=x[0]) for x in list_of_dataset_name_and_ext_feature_df_csv_file_paths], ignore_index=True)
assert 'numbered_donor_id' not in list(all_ext_donor_feature_df.columns)
all_ext_donor_feature_df = sc_rna_seq_preprocessing_params.get_df_with_numbered_donor_id(all_ext_donor_feature_df)
mds_analysis.add_non_ARCH4_sample_in_donor_with_any_inconsistent_ARCH_col(all_ext_donor_feature_df)
treatment_df = clinical_data_interface_and_utils.get_nili_treatment_df()
if 'treatment' in all_ext_donor_feature_df.columns:
    assert all_ext_donor_feature_df['treatment'].isna().all()
    all_ext_donor_feature_df.drop(columns='treatment', inplace=True)
all_ext_donor_feature_df = generic_utils.merge_preserving_df1_index_and_row_order(all_ext_donor_feature_df, treatment_df, on=['donor_id', 'bleeding_date'])
sc_rna_seq_preprocessing_params.add_latest_diagnosis_class_col(all_ext_donor_feature_df)
all_ext_donor_feature_df = clinical_data_interface_and_utils.add_donor_closest_cbc_cols(all_ext_donor_feature_df)
clinical_df = clinical_data_interface_and_utils.get_minimal_clinical_data_df()
all_ext_donor_feature_df = clinical_data_interface_and_utils.add_donor_closest_bm_facs_blast_cols(all_ext_donor_feature_df, clinical_df)

if 0:
    all_ext_feature_df = pd.read_csv(get_mds_params()['all_ext_donor_feature_df_csv_file_path'])
    mds_analysis.generate_blood_sample_id_df(all_ext_feature_df).to_csv(dummy_for_safety, index=False)
    mds_analysis.generate_scrna_exp_id_df(all_ext_feature_df).to_csv(dummy_for_safety, index=False)
    raise
all_ext_donor_feature_df = generic_utils.merge_preserving_df1_index_and_row_order(
    all_ext_donor_feature_df, pd.read_csv(get_sc_rna_seq_preprocessing_params()['blood_sample_id_df_csv_file_path']))
all_ext_donor_feature_df = generic_utils.merge_preserving_df1_index_and_row_order(
    all_ext_donor_feature_df, pd.read_csv(get_sc_rna_seq_preprocessing_params()['scrna_exp_id_df_csv_file_path']))
all_ext_donor_feature_df['nimrod_compatible_donor_id'] = all_ext_donor_feature_df['donor_id'].replace(
    get_sc_rna_seq_preprocessing_params()['240723_donor_id_i_use_to_nimrod_donor_id'])
mds_analysis.add_unreliable_arch_cols_and_remove_vaf_vals(all_ext_donor_feature_df)

all_ext_donor_feature_df.sort_values(['blood_sample_id', 'scrna_exp_id'], inplace=True)

all_misc_numbered_donor_info = mds_analysis.get_all_misc_numbered_donor_info()

all_numbered_donor_df = all_misc_numbered_donor_info['numbered_donor_df']
assert 'numbered_donor_id' not in list(all_numbered_donor_df.columns)
assert len(all_numbered_donor_df) == len(all_ext_donor_feature_df)
all_numbered_donor_df = generic_utils.merge_preserving_df1_index_and_row_order(
    all_numbered_donor_df, all_ext_donor_feature_df[['exp_name', 'donor_id', 'numbered_donor_id', 'dataset_name', 'biased_composition_due_to_discarded_low_umi_count_barcodes', 'known_treatment', 'latest_diagnosis_class']], on=['exp_name', 'donor_id'])
all_numbered_donor_df['numbered_donor_id'] = all_numbered_donor_df['numbered_donor_id'].astype(str)

assert (all_numbered_donor_df['donor_age'] >= 0).all()
assert (all_ext_donor_feature_df['donor_age'] >= 0).all()

all_ext_donor_feature_df = mds_analysis.get_df_with_usage_in_analysis_masks(all_ext_donor_feature_df)
mds_analysis.add_mean_near_neighbor_dist(all_ext_donor_feature_df)

all_numbered_donor_df.to_csv(get_mds_params()['all_numbered_donor_df_csv_file_path'], index=False)
all_ext_donor_feature_df.to_csv(get_mds_params()['all_ext_donor_feature_df_csv_file_path'], index=False)